In [53]:
import numpy as np
import pandas as pd
import tensorflow as tf



In [54]:
df_user = pd.read_csv('../../dataset/df_user.csv')
place_num = len(df_user.Place_Id.unique())
n_users = len(df_user.User_Id.unique())



In [55]:
df_destination = pd.read_csv('../../dataset/new_bali_dataset.csv', delimiter=';', header=0)
df_destination['price'].fillna(0, inplace=True)
df_destination['category'].fillna('unknown', inplace=True)

In [56]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df_user, test_size=0.2, random_state=42)

In [57]:
def create_model():
    # creating destination embedding path
    destination_input = Input(shape=[1])
    destination_embedding = Embedding(place_num+1, 5)(destination_input)
    destination_flat = Flatten()(destination_embedding)

    # creating user embedding path
    user_input = Input(shape=[1])
    user_embedding = Embedding(n_users+1, 5)(user_input)
    user_flat = Flatten()(user_embedding)

    # concatenate features
    conc = Concatenate()([destination_flat, user_flat])

    # add fully-connected-layers
    x = Dense(128, activation='relu')(conc)
    x = Dense(64, activation='relu')(x)
    output = Dense(1)(x)

    # Create model and compile it
    model = Model([user_input, destination_input], output)
    model.compile('adam', 'mean_squared_error')
    history = model.fit([train.User_Id, train.Place_Id], train.Place_Ratings, epochs=20)
    return history, model

Predict

In [58]:
# bali_dataset = pd.read_csv('../../dataset/new_bali_dataset.csv', delimiter=';')
# recommended_places = bali_dataset.loc[recommended_tourism_ids, 'place']
# recommended_places

In [84]:
history, model = create_model()
model.save('../model.h5')

Epoch 1/20
229/229 [==============================] - 4s 6ms/step - loss: 3.1111
Epoch 2/20
229/229 [==============================] - 1s 5ms/step - loss: 1.8728
Epoch 3/20
229/229 [==============================] - 1s 4ms/step - loss: 1.8360
Epoch 4/20
229/229 [==============================] - 1s 5ms/step - loss: 1.8242
Epoch 5/20
229/229 [==============================] - 1s 4ms/step - loss: 1.8093
Epoch 6/20
229/229 [==============================] - 1s 4ms/step - loss: 1.8136
Epoch 7/20
229/229 [==============================] - 1s 3ms/step - loss: 1.8045
Epoch 8/20
229/229 [==============================] - 1s 5ms/step - loss: 1.7982
Epoch 9/20
229/229 [==============================] - 2s 7ms/step - loss: 1.7928
Epoch 10/20
229/229 [==============================] - 1s 4ms/step - loss: 1.7915
Epoch 11/20
229/229 [==============================] - 1s 3ms/step - loss: 1.7730
Epoch 12/20
229/229 [==============================] - 1s 3ms/step - loss: 1.7459
Epoch 13/20
229/229 [====

c:\Users\u\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [77]:
def create_final_data(list_of_idx):
    new_df = df_destination[df_destination.index.isin(list_of_idx)].copy()
    list_of_dest = []

    for row in new_df.iterrows():
        dest_dict = {}
        dest_dict['place'] = row[1]['place']
        dest_dict['url'] = row[1]['url']
        dest_dict['address'] = row[1]['address']
        dest_dict['is_accessibility'] = row[1]['is_accessibility']
        dest_dict['rating'] = row[1]['rating']
        dest_dict['n_reviews'] = row[1]['n_reviews']
        dest_dict['price'] = row[1]['price']
        dest_dict['category'] = row[1]['category']
        dest_dict['description'] = row[1]['description']
        list_of_dest.append(dest_dict)
    
    return list_of_dest

In [85]:
model.save('../model.h5')
model = tf.keras.models.load_model('../model.h5')


In [83]:
def might_like(idx_user, model=model):
    tourism_data = np.array(list(set(df_user.Place_Id)))
    user = np.array([idx_user]*len(tourism_data))

    
    predictions = model.predict([user, tourism_data])
    predictions = np.array([a[0] for a in predictions])
    
    recommended_tourism_idx = (-predictions).argsort()[:10]
    
    #convert index to data place
    data = create_final_data(recommended_tourism_idx)
    return data

In [79]:
idx_user = 5
get_recom = might_like(idx_user)
get_recom

7/7 [==============================] - 0s 3ms/step


[{'place': 'Kedungu beach Bali',
  'url': 'https://www.google.com/maps/place/Kedungu+beach+Bali/@-8.7226249,114.592745,10z/data=!4m11!1m3!2m2!1spantai+di+bali!6e1!3m6!1s0x2dd2377ae58eba1b:0x8665f5f539b81682!8m2!3d-8.6088824!4d115.083265!15sCg5wYW50YWkgZGkgYmFsaVoQIg5wYW50YWkgZGkgYmFsaZIBBWJlYWNo4AEA!16s%2Fg%2F11b6hlpmbp?entry=ttu',
  'address': 'Jl. Pantai Kedungu, Desa Belalang, Belalang, Kediri, Tabanan Regency, Bali',
  'is_accessibility': 0,
  'rating': 4.4,
  'n_reviews': 1817,
  'price': 15000,
  'category': 'Pantai',
  'description': 'A beach with waves suitable for surfing, Kedungu Beach also offers stunning sunset views. The beach is frequented by surfers and visitors who want to enjoy the relaxed and natural atmosphere of the beach.'},
 {'place': 'Pantai Ciwa',
  'url': 'https://www.google.com/maps/place/Pantai+Ciwa/@-8.438572,114.84591,17z/data=!3m1!4b1!4m6!3m5!1s0x2dd3d30fee57acbd:0xb2f8b6dc49b14c5f!8m2!3d-8.4385773!4d114.8484849!16s%2Fg%2F11r9h0yl2l?entry=ttu',
  'address'